# Output Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine.

Authors: Johannes Giehl (jfg.eco@cbs.dk), Dana J. Hentschel (djh.eco@cbs.dk), Lucia Ciprian (luc.eco@cbs.dk)

## General settings

### Packages:

In [1]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
import sys
from nord_h2ub_hexhex_output import *

### Check whether SpineOpt has run correctly

SpineToolbox has per default a certain time limit on each optimization window. If it reaches that time limit, a suboptimal solution is reported. The following code checks whether this time limit is reached in at least one optimization step and kills the console if necessary.

In [2]:
directory = get_base_path()
print(directory)

C:\Users\luc.eco\OneDrive - CBS - Copenhagen Business School\Documents\GitHub\Nord_H2ub


In [3]:
# Get spineopt logs
parent_folder_time = os.path.join(directory, "Spine_Projects", "02_basic_energy_model", ".spinetoolbox", "items", "run_spineopt", "logs")

if not os.path.exists(parent_folder_time):
    print(f"The directory '{parent_folder_time}' does not exist.")

# Filter for log files in the folder
log_files = [l for l in os.listdir(parent_folder_time) if l.endswith('.log') and os.path.isfile(os.path.join(parent_folder_time, l))]

if not log_files:
    print('\033[1m' + '\033[91m' + 'No log files found, check your SpineOpt settings. Make sure you enable "Log process output to a file" in SpineToolbox.')
    sys.exit(1)
else:
    latest_log = max(log_files, key=lambda x: os.path.getmtime(os.path.join(parent_folder_time, x)))
    latest_log_path = os.path.join(parent_folder_time, latest_log)
    
    with open(latest_log_path, 'r') as file:
        numbers = []
        for line in file:
            if "Timing" in line:
                match = re.search(r"Timing\s+([\d.]+)", line)
                if match:
                    number = float(match.group(1))
                    if number > 3000:
                        print('\033[1m' + '\033[91m' + 'Warning, at least one of the time windows has reached the maximum! Results are not reliable anymore.')
                        sys.exit(1)

### Parameters

In [4]:
# Get current working directory:
cwd = os.getcwd()

# Adjust path to get parameters:
parameter_path = os.path.join(os.path.dirname(os.path.dirname(cwd)), "01_input_data\\00_functions")
with open(os.path.join(parameter_path, 'parameters.pkl'), 'rb') as file:
    parameters = pickle.load(file)

# Extract variables of interest
run_name = f"{latest_log[:15]}_{parameters['run_name']}"
time_horizon = parameters['lcoe_years']
wacc = parameters['wacc']
starting_year = parameters['starting_year']
simulation_year = parameters['year']  # Year of used electricity prices 
fuel = parameters['product']

In [5]:
#FOR TESTING, value in .pkl file is not what we used in our paper
#delete after testing
time_horizon = 20
#FOR TESTING, DELETE AFTER
#run_name = "TEST"

In [6]:
#define if the model should use the last spine optimization results or a specific file
last_model_run = False   # Set to False if specific file is needed 

#set names of specific files if needed
#run_name = 'base_case'
specific_file_name = 'Output_exported_' + run_name +'.xlsx'

#output files of this script
#file names to store the prepared output
output_prepared_export = 'Output_exported_'+ run_name +'.xlsx'

### File paths:

In [7]:
#get path of latest spine results
#parent folder
parent_folder_results = os.path.join(directory, "Spine_Projects", "02_basic_energy_model", ".spinetoolbox", "items", "exporter", "output")

folders = [f for f in os.listdir(parent_folder_results) if os.path.isdir(os.path.join(parent_folder_results, f))]
if not folders:
    print("No folders found.")
else:
    latest_folder = max(folders, key=lambda x: os.path.getmtime(os.path.join(parent_folder_results, x)))
    #latest_folder = "run@2024-10-14T13.04.57"
    latest_folder_path = os.path.join(parent_folder_results, latest_folder)
latest_folder_path = latest_folder_path.replace('\\', '/')
folder_path_results = latest_folder_path
folder_path_results += '/'

#get the information of the prepared input data that is used for the spine optimization
prepared_input_file_path = os.path.join(directory, "Spine_Projects", "01_input_data", "02_input_prepared")

#path to files from specific runs
path_specific_runs = '02_runs_EURO/01_output_raw/'
#prepared output data export to
output_file_path = os.path.join(directory, "Spine_Projects", "03_output_data", "01_basic_energy_model_outputs")

In [8]:
#Input files for this script

#file name export from SpineToolbox
output_exported_file = 'Output_exported@Exporter.xlsx'
#input file used for the optimization
data_from_inputs = 'methanol_Input_prepared.xlsx'
#this way of the output data preparation must be changed
#the information is manually added to the input xlsx and does not exist in the automated input generation
data_from_inputs_temporary = 'methanol_Input_prepared_for_output_temporary.xlsx'

In [9]:
## for testing, comparison with paper results DELETE AFTER TESTING 
prepared_input_file_path = os.path.join(directory, "Spine_Projects", "01_input_data", "02_input_prepared", "paper_energy_2024_inputs")
data_from_inputs = 'paper_energy_2024_input_base_10op.xlsx'
path_specific_runs =os.path.join(directory, "Spine_Projects", "03_output_data", "03_runs_paper_energy_2024","01_output_raw")
specific_file_name = 'Output_exported_paper_energy_2024_base_10op.xlsx'

In [10]:
#combine input path and files

#to input files
full_path_data_from_inputs = os.path.join(prepared_input_file_path, data_from_inputs)

#to specific files
full_path_specific_files = os.path.join(path_specific_runs, specific_file_name)
#for temporary appraoch
full_path_data_from_inputs_temporary  = os.path.join(prepared_input_file_path, data_from_inputs_temporary)

#for output_data
full_path_export_output_data = os.path.join(output_file_path, output_prepared_export)

## Data preparation

### Data Import

In [11]:
if last_model_run == True:
    df_output_raw = pd.read_excel(os.path.join(folder_path_results + output_exported_file), sheet_name=-1)
else:
    df_output_raw = pd.read_excel(full_path_specific_files)

df_PV_prices = pd.read_excel(full_path_data_from_inputs, sheet_name='Energy_prices')
def_params = df_PV_prices = pd.read_excel(full_path_data_from_inputs, sheet_name='Definition_parameters')
con_inv_params = pd.read_excel(full_path_data_from_inputs, sheet_name="Connection_Inv_Parameters")
unit_inv_params = pd.read_excel(full_path_data_from_inputs, sheet_name="Unit_Inv_Parameters")
stor_inv_params = pd.read_excel(full_path_data_from_inputs, sheet_name="Storage_Inv_Parameters")
nodes_params = pd.read_excel(full_path_data_from_inputs, sheet_name="Nodes")
model_params = pd.read_excel(full_path_data_from_inputs, sheet_name="Model")
capacity_params = pd.read_excel(full_path_data_from_inputs, sheet_name="Object__to_from_node")
print("Done")

Done


### Data Frame Preparation

In [12]:
#create a copy of the original output DataFrame
df_output = df_output_raw.copy()

# Replace NaN values with empty strings in the first three rows
df_output.iloc[:3] = df_output.iloc[:3].fillna('')

# Combine the old header with the strings from the first three rows for each column
new_headers = df_output.columns + '_' + df_output.iloc[0] + '_' + df_output.iloc[1] + '_' + df_output.iloc[2]
old_headers = df_output.columns

# Set the new headers
df_output.columns = new_headers

# Drop the first three rows
#might be helpful bot not implemented now
#df_output = df_output.drop([0, 1, 2])

# Reset the index
df_output.reset_index(drop=True, inplace=True)
# Rename the first column to "timeseries"
df_output.columns.values[0] = "timeseries"

### Data Adjustments

In [13]:
#calculate revenues from PV sales on the wholesale market
selected_column_name = None
for column_index in range(len(df_output.columns)):
    if (df_output.iloc[0, column_index] == 'pl_wholesale' and 
        df_output.iloc[1, column_index] == 'to_node' and df_output.iloc[2, column_index] == 'power_wholesale'):         
        selected_column_name = df_output.columns[column_index]
        break
# If connected to grid PV power is sold, else revenue is 0
if 'power_wholesale_out' in df_PV_prices.columns:                 #To adjust
    if selected_column_name:
        df_output['Revenue_from_PV'] = df_output[selected_column_name].iloc[3:] * df_PV_prices['power_wholesale_out'].iloc[4:]          
    else:
        print("Column with specified headers not found in output.")

else: 
    df_output['Revenue_from_PV'] = 0 


In [14]:
# Subtract slack penalty if any (Penalty costs assumed to be equal for all nodes)
# Calculate the slacks
node_slack_neg = df_output.filter(like='node_slack_neg')
node_slack_pos = df_output.filter(like='node_slack_pos')

node_slack_neg_numeric = node_slack_neg.apply(pd.to_numeric, errors='coerce')
node_slack_pos_numeric = node_slack_pos.apply(pd.to_numeric, errors='coerce')

node_slack_neg_sum = node_slack_neg_numeric.sum().sum()
node_slack_pos_sum = node_slack_pos_numeric.sum().sum()

total_slack = node_slack_neg_sum + node_slack_pos_sum
# node slack penalty
slack_penalty = nodes_params['node_slack_penalty'][(nodes_params['node_slack_penalty'] != 0)&(nodes_params['node_slack_penalty'].notna())].iloc[0]

# total penalty that had to be paid 
total_slack_penalty = total_slack*slack_penalty

In [24]:
# Determining the fom cost of the RES if the optimization was run without investing in cost PV, OffW or OnW
RES = ["solar_plant", "wind_onshore", "wind_offshore"]
# Create dataframe of object_invested for each object (needed later as well)
objects_invested_df = df_output.filter(like='_invested')
objects_invested_df.iloc[4:] = objects_invested_df.iloc[4:].apply(pd.to_numeric, errors='coerce')


RES_fom_costs = {}
for col_name, col_data in objects_invested_df.items():
    if col_data.iloc[0] in RES:
        RES_invested_during = col_data[3:].loc[(col_data[3:] != 0) & (col_data[3:].notna())].sum()
        if RES_invested_during==0:
            RES_max_cap = capacity_params.loc[capacity_params.loc[:,"Object_name"]==col_data.iloc[0], "Value"].values[0]
            RES_installed_cap = RES_invested*RES_max_cap
            RES_invested_before = unit_inv_params.loc[unit_inv_params.iloc[:, 0] == col_data.iloc[0],"initial_units_invested_available"].values[0]
            fom_per_MW = def_params.loc[def_params.iloc[:, 0] == col_data.iloc[0], "Value"].values[0]       
            RES_fom_costs[col_data.iloc[0]] = fom_per_MW * RES_installed_cap
        else:
            RES_fom_costs[col_data.iloc[0]] = 0 
    extra_fom_costs = sum(RES_fom_costs.values())

    
#get total cost of the system
total_costs = df_output.filter(like='total_cost')
total_costs = total_costs.apply(pd.to_numeric, errors='coerce')
total_costs = total_costs.sum().sum()
total_costs += total_slack_penalty
total_costs += extra_fom_costs

#get total revenue from PV power sale (times -1 is relevant as the input is structured that negative prices for exports reduce total cost). 
total_PV_revenue = df_output['Revenue_from_PV'].sum()*(-1)
#calculate cost without PV revenue
adjusted_costs = total_costs + total_PV_revenue 
                            
#create separate DataFrame for total and adjusted cost
df_system_cost_output = pd.DataFrame({
    'Total_cost': [total_costs],
    'PV_revenue': [total_PV_revenue],
    'Total_adjusted_cost': [adjusted_costs]
})

C:\Users\luc.eco\AppData\Local\Temp\ipykernel_13664\3165232250.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  objects_invested_df.iloc[4:] = objects_invested_df.iloc[4:].apply(pd.to_numeric, errors='coerce')


In [207]:
# Identify columns to drop
columns_to_drop_1 = df_output.filter(like='total_costs').columns
# Drop the identified columns if any are found
if not columns_to_drop_1.empty:
    df_output.drop(columns=columns_to_drop_1, inplace=True)

#test this and implement an if check
columns_to_drop_2 = df_output.filter(like='unit_flow_op').columns
# Drop the identified columns if any are found
if not columns_to_drop_2.empty:
    df_output.drop(columns=columns_to_drop_2, inplace=True)

## Calculate LCOE

### Calculate Investment Cost

In [208]:
dict_investments = {}
connection_costs = 0
unit_costs = 0
storage_costs = 0
## Check number of slices of roll forward
for line_number, line in model_params.iterrows():
    if line["Parameter"] == "roll_forward":
        number_str = ''.join([char for char in line["Value"] if char.isdigit()])
        number = int(number_str)
        roll_forward_slices = 8760//number
        break
    else:
        roll_forward_slices = 1


# Create dictionary with total number of object_invested for each object
objects_invested_dict = {}
for col_name, col_data in objects_invested_df.items():
    objects_invested_dict[col_data.iloc[0]] = col_data[3:].loc[(col_data[3:] != 0) & (col_data[3:].notna())].sum()


# Convert lifetime of '365D' for example to 365
unit_inv_params["unit_investment_econ_lifetime"] = unit_inv_params["unit_investment_econ_lifetime"].astype(str).str.replace('D', '', regex=False).astype(float)
con_inv_params["connection_investment_econ_lifetime"] = con_inv_params["connection_investment_econ_lifetime"].astype(str).str.replace('D', '', regex=False).astype(float)
stor_inv_params["storage_investment_econ_lifetime"] = stor_inv_params["storage_investment_econ_lifetime"].astype(str).str.replace('D', '', regex=False).astype(float)


# Adding investemnt costs of intially investested objects
for row_number_con, row_con in con_inv_params.iterrows():
    yearly_iv_cost = row_con["connection_investment_cost"]* roll_forward_slices   # Investment costs for full capacity and one year
    lifetime = row_con["connection_investment_econ_lifetime"]/365                 # Lifetime of object in years
    invested_available = row_con["initial_connections_invested_available"]        # Number of objects initially available for investements 
    if row_con["Object_name"] in objects_invested_dict:
        invested_opt = objects_invested_dict[row_con["Object_name"]]              # Number of objects invested in during optimization
    else: 
        invested_opt = 0
    invested_total = invested_available + invested_opt
    connection_costs += (invested_total * yearly_iv_cost * lifetime)              # Total investment costs for connections
    
for row_number_unit, row_unit in unit_inv_params.iterrows():
   
    yearly_iv_cost = row_unit["unit_investment_cost"]* roll_forward_slices
    lifetime = row_unit["unit_investment_econ_lifetime"]/365
    invested_available = row_unit["initial_units_invested_available"]
    if row_unit["Object_name"] in objects_invested_dict:
        invested_opt = objects_invested_dict[row_unit["Object_name"]]
    else: 
        invested_opt = 0
    invested_total = invested_available + invested_opt
    unit_costs += (invested_total * yearly_iv_cost * lifetime)
    
for row_number_stor, row_stor in stor_inv_params.iterrows():
    yearly_iv_cost = row_stor["storage_investment_cost"]* roll_forward_slices
    lifetime = row_stor["storage_investment_econ_lifetime"]/365
    invested_available = row_stor["initial_storages_invested"]
    if row_stor["Object_name"] in objects_invested_dict:
        invested_opt = objects_invested_dict[row_stor["Object_name"]]
    else: 
        invested_opt = 0
    invested_total = invested_available + invested_opt
    storage_costs += (invested_total * yearly_iv_cost * lifetime)
    
dict_investments["Connection_Investement_Costs"]  = connection_costs
dict_investments["Storage_Investement_Costs"]  = storage_costs
dict_investments["Unit_Investement_Costs"]  = unit_costs


total_investment = sum(dict_investments.values())

C:\Users\luc.eco\AppData\Local\Temp\ipykernel_28904\776918219.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  objects_invested_df.iloc[4:] = objects_invested_df.iloc[4:].apply(pd.to_numeric, errors='coerce')


### variable costs

In [209]:
#get annual costs
#annual_costs = df_system_cost_output.loc['total_costs_toy__', 'Total_adjusted_cost'] - total_slack
annual_costs = adjusted_costs-total_slack
#annual cost including PV revenue
#annual_costs_with_PV = df_system_cost_output.loc['total_costs_toy__', 'Total_cost'] - total_slack
annual_costs_with_PV = total_costs-total_slack

### energy output

In [210]:
#energy output
energy_output = df_output.filter(like='pl_ch3oh_demand_from_node_ch3oh')         

# Convert strings to numbers, ignoring non-numeric values (relevant as first rows are strings)
energy_output_value = pd.to_numeric(energy_output.iloc[:,0], errors='coerce').sum()

In [211]:
#calculation of the present value factor
pcf_value = present_value_factor(time_horizon, wacc)
#annual_costs_with_PV = 6659285.68590276
LCOE = (total_investment + (annual_costs * pcf_value)) / (energy_output_value * pcf_value)
LCOE_PV = (total_investment + (annual_costs_with_PV * pcf_value)) / (energy_output_value * pcf_value)

In [212]:
#create a pandas data frame with all LCOE information

LCOE_GJ = LCOE/3.6
LCOE_t = LCOE_GJ*19.9

LCOE_GJ_PV = LCOE_PV/3.6
LCOE_PV_t = LCOE_GJ_PV*19.9

energy_output_value_t = (energy_output_value * 3.6) / 19.9


# Create a dictionary with the parameters
data_LCOE = {
    'LCOE [Euro/MWh]': LCOE,
    'LCOE [Euro/t]': LCOE_t,
    'total_investment': total_investment,
    'annual_costs': annual_costs,
    'energy_production [MWh]': energy_output_value,
    'energy_production [t]': energy_output_value_t,
    'pcf_value': pcf_value,
    'run_name': run_name
}

data_LCOE_PV = {
    'LCOE [Euro/MWh]': LCOE_PV,
    'LCOE [Euro/t]': LCOE_PV_t,
    'total_investment': total_investment,
    'annual_costs': annual_costs_with_PV,
    'energy_production [MWh]': energy_output_value,
    'energy_production [t]': energy_output_value_t,
    'pcf_value': pcf_value,
    'run_name': run_name + '_PV'
}

# Convert the dictionary to a DataFrame and set the index to the run name
df_LCOE_information = pd.DataFrame([data_LCOE])
df_LCOE_information.set_index('run_name', inplace=True)

df_LCOE_PV_information = pd.DataFrame([data_LCOE_PV])
df_LCOE_PV_information.set_index('run_name', inplace=True)

df_LCOE = pd.concat([df_LCOE_information, df_LCOE_PV_information])

### Carbon price calculation

In [213]:
# Determine the year of calculations. If the simulated year is not between 2019 and 2024, 2019 is used for calculations 
# as this year's energy data was not uncommonly affected by pandemic or political events 
if simulation_year >=2019 and simulation_year<=2024: 
    year = simulation_year
else:
    year = 2019

## Extract corresponding data of used product

if fuel == 'methanol':
    # fossil based data
    energy_density = 19.7*10**9                               # J/t  (source: https://ebookcentral.proquest.com/lib/kbhnhh-ebooks/detail.action?docID=5116604)   
    co2_indstr_high = 2.863                                   # kg*CO2/kg*MeOH   (source: https://www.sciencedirect.com/science/article/pii/S0360319922002415)
    co2_indstr_low = 2.05                                     # kg*CO2/kg*MeOH   (source: https://www.methanol.org/wp-content/uploads/2022/01/CARBON-FOOTPRINT-OF-METHANOL-PAPER_1-31-22.pdf)
    co2_indstr_perJ = 103 * 10**(-9)                          # kg*CO2/J  (source: https://www.methanol.org/wp-content/uploads/2022/01/CARBON-FOOTPRINT-OF-METHANOL-PAPER_1-31-22.pdf)

    # costs ([$/t]) of indutrial equivalent for years 2019 to 2024 in dictionary
    costs_list = [370,205,395,399,321.5,374]                  # $/t (source: https://shipandbunker.com/prices/emea/nwe/nl-rtm-rotterdam#MEOH)
    indstr_costs = dict(zip(list(range(2019,2025)), costs_list))

elif fuel == 'ammonia':
    # fossil based data
    energy_density = 22.5*10**9                              # J/t  (source: https://www.sciencedirect.com/science/article/pii/S0360128517302320)   
    co2_indstr_low = 2.55                                    # kg*CO2/kg*NH3   (source: https://pubs.rsc.org/en/content/articlelanding/2020/gc/d0gc02301a) 
    co2_indstr_high = 2.6                                    # kg*CO2/kg*NH3   (source: https://pubs.rsc.org/en/content/articlelanding/2022/gc/d2gc00843b/unauth) 

    # costs ([$/t]) of indutrial equivalent for years 2019 to 2024 in dictionary
    costs_list = [366.67,307.5,564.17,1240.83,670,584.17]    # $/t (source: https://businessanalytiq.com/procurementanalytics/index/ammonia-price-index/)
    indstr_costs = dict(zip(list(range(2019,2025)), costs_list))
    
elif fuel == 'diesel':
    # fossil based data
    energy_density = 42.64*10**9                                # J/t  (source: https://www.chevron.com/-/media/chevron/operations/documents/diesel-fuel-tech-review.pdf?utm_source=chatgpt.com) 
    co2_indstr_high = 3.79                                      # kg*CO2/kg*MeOH   (source: https://www.sciencedirect.com/science/article/pii/S1364032122000867) 
    co2_indstr_low = 2                                       # kg*CO2/kg*MeOH   (source: DUMMY VALUE) 

    # costs ([$/t]) of indutrial equivalent for years 2019 to 2024 in dictionary
    costs_list = [222,222,222,222,222,222]                   # $/t (source: DUMMY VALUES)
    indstr_costs = dict(zip(list(range(2019,2025)), costs_list))
    
elif fuel == 'egasoline':
    # fossil based data
    energy_density = 44*10**9                                # J/t  (source: PDF of https://ens.dk/en/analyses-and-statistics/technology-data-renewable-fuels)   
    co2_indstr_high = 3.78                                   # kg*CO2/kg*MeOH   (source: https://www.sciencedirect.com/science/article/pii/S1364032122000867) 
    co2_indstr_low = 2                                       # kg*CO2/kg*MeOH   (source: DUMMY VALUE) 

    # costs ([$/t]) of indutrial equivalent for years 2019 to 2024 in dictionary
    costs_list = [222,222,222,222,222,222]                   # $/t (source: DUMMY VALUES)
    indstr_costs = dict(zip(list(range(2019,2025)), costs_list))

elif fuel == 'hydrogen':
    # fossil based data
    energy_density = 120*10**9                               # J/t  (source: https://pubs.acs.org/doi/full/10.1021/jp406523w)   
    co2_indstr_high = 13                                     # kg*CO2/kg*MeOH   (source: https://www.sciencedirect.com/science/article/pii/S0360319922052570) 
    co2_indstr_low = 10.5                                    # kg*CO2/kg*MeOH   (source: https://www.sciencedirect.com/science/article/pii/S0360319922052570) 

    # costs ([$/t]) of indutrial equivalent for years 2019 to 2024 in dictionary
    # Grey hydrogen (data starts in 2021, replace 2019 and 2020 rice when found)
    costs_list = [4053,4053,4053,6870,3783,3348]             # $/t (source: https://businessanalytiq.com/procurementanalytics/index/grey-hydrogen-price-index/)
    indstr_costs = dict(zip(list(range(2019,2025)), costs_list))

elif fuel == 'jet_fuel':
    # fossil based data
    energy_density = 1*10**9                                 # J/t  (source: DUMMY VALUE)   
    co2_indstr_high = 2                                      # kg*CO2/kg*MeOH   (source: DUMMY VALUE) 
    co2_indstr_low = 2                                       # kg*CO2/kg*MeOH   (source: DUMMY VALUE) 

    # costs ([$/t]) of indutrial equivalent for years 2019 to 2024 in dictionary
    costs_list = [222,222,222,222,222,222]                   # $/t (source: DUMMY VALUES)
    indstr_costs = dict(zip(list(range(2019,2025)), costs_list))

elif fuel == 'synthetic_methane_gas':
    # fossil based data
    energy_density = 55.47*10**9                             # J/t  (source: https://www.sciencedirect.com/science/article/pii/S1385894715002971)   
    co2_indstr_high = 36                                     # kg*CO2/kg*MeOH   (source: https://pubs.rsc.org/en/content/articlehtml/2018/em/c8em00414e) 
    co2_indstr_low = 27.9                                    # kg*CO2/kg*MeOH   (source: https://www.sciencedirect.com/science/article/pii/S1462901122001204) 

    # costs ([$/t]) of indutrial equivalent for years 2019 to 2024 in dictionary
    costs_list = [222,222,222,222,222,222]                   # $/t (source: DUMMY VALUES)
    indstr_costs = dict(zip(list(range(2019,2025)), costs_list))

In [214]:
## Fuel independent data 

# Average currency exhange rates([€/$]) for years 2019 to 2024 in dictionary
exchange_list =  [0.8932, 0.8602, 0.8945, 0.9371, 0.9248, 0.9239]                    # €/$    (source: https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-usd.en.html)
exchange_rates = dict(zip(list(range(2019,2025)), exchange_list))

# European allowances 2019-2024, 2050
EUA_list = [24.723125, 24.38660287, 54.15156951, 80.18404545, 83.59650224, 64.75701] # €/(t*CO2-eq)  (source: https://icapcarbonaction.com/en/ets-prices)
avg_EUA = dict(zip(list(range(2019,2025)), EUA_list))

# Average Emissions of grid power 2019-2024 (!! Udate value of 2024 when available on https://energinet.dk/eldeklaration/!!)
electr_emissions_list = [372/(10**6), 478.5/(10**6), 411.5/(10**6), 432/(10**6), 499.8/(10**6), 450/(10**6)] # kg Co2/(Wh electricity)  (source: https://energinet.dk/media/1duf3x0o/19_07249-24-generel-eldeklaration-historik-10536640_5986882_0.xlsx)
avg_electr_emissions = dict(zip(list(range(2019,2025)), electr_emissions_list))

# Yearly demand
demand_yearly_t = energy_output_value_t


In [215]:
## Result data of simulations
# Determine produced PV power and sold and bought grid electricity
Production_PV_df = df_output.filter(like='solar_plant_to_node_power')
Production_PV = sum(Production_PV_df.iloc[4:, 0].tolist())

El_from_grid_df = df_output.filter(like='wholesale_to_node_power') 
El_from_grid = sum(El_from_grid_df.iloc[4:, 0].tolist())
El_to_grid_df = df_output.filter(like='wholesale_from_node_power') 
El_to_grid = sum(El_to_grid_df.iloc[4:, 0].tolist())
costs_hub = LCOE_PV_t

In [216]:
## Calculations 

# Electricity used for production, PV share and used grid electricity in production 
El_used = El_from_grid+Production_PV- El_to_grid                                                                 # MWh 
PV_used_rel = (Production_PV-El_to_grid)/El_used                                                                 # % 
el_demand_grid = El_from_grid*(1-PV_used_rel)                                                                    # MWh

# Electricity taken from the grid 
grid_electricity = el_demand_grid*10**6                                                                          # Wh

# Emissions per year 
emissions_hub = grid_electricity*avg_electr_emissions[year]/1000                                                 # t Co2
emissions_indstr_low = demand_yearly_t*co2_indstr_low                                                            # t Co2
emissions_indstr_high = demand_yearly_t*co2_indstr_high                                                          # t Co2

# Emissions per tonne of fuel
emissions_per_tonne = emissions_hub/demand_yearly_t                                                              # t*Co2/t*MeOH

# Costs for yearly demand of fuel (produced in hub vs. produced industrial)
ym_costs = costs_hub*demand_yearly_t                                                                             # €
ym_costs_indstr = indstr_costs[year]*exchange_rates[year]*demand_yearly_t                                        # €

# Calculations of neccessary carbon prices   (extra costs per saved tonne of co2)

# Comparison with lower co2 emissions of industrial methanol 
CPl = (ym_costs-ym_costs_indstr)/(emissions_indstr_low-emissions_hub)                                            # €/t*Co2

# Comparison with higher co2 emissions of industrial methanol 
CPh = (ym_costs-ym_costs_indstr)/(emissions_indstr_high-emissions_hub)                                           # €/t*Co2

# Comparison in case of purely green E-Methanol with lower co2 industrial methanol
CPl_green = (ym_costs-ym_costs_indstr)/(emissions_indstr_low)                                                    # €/t*Co2

# Comparison in case of purely green E-Methanol with higher co2 industrial methanol
CPh_green = (ym_costs-ym_costs_indstr)/(emissions_indstr_high)                                                   # €/t*Co2


In [217]:
## Create dataframe with data for CO2 price and CO2 equivalents 
data_carbon_price = {
    "Scenario": [
        "Hub fuel comp. to indstrl. (lower emission)",
        "Hub fuel comp. to indstrl. (higher emission)",
        "CO2 neutral fuel comp. to indstrl. (lower emission)",
        "CO2 neutral fuel comp. to indstrl. (higher emission)"
    ],
    "CO2 Equivalent (Greener Fuel)": [emissions_per_tonne, emissions_per_tonne, 0, 0],
    "CO2 Equivalent (Fossil Fuel)": [co2_indstr_low, co2_indstr_high, co2_indstr_low, co2_indstr_high],
    "Carbon Price": [CPl, CPh, CPl_green, CPh_green]
}

df_carbon_price = pd.DataFrame(data_carbon_price)
df_carbon_price

,Scenario,CO2 Equivalent (Greener Fuel),CO2 Equivalent (Fossil Fuel),Carbon Price
0,Hub fuel comp. to indstrl. (lower emission),1.340479,2.050,1114.309881
1,Hub fuel comp. to indstrl. (higher emission),1.340479,2.863,519.287566
2,CO2 neutral fuel comp. to indstrl. (lower emis...,0.000000,2.050,385.671314
3,CO2 neutral fuel comp. to indstrl. (higher emi...,0.000000,2.863,276.153054


### Creating one combined excel and export

In [234]:
with pd.ExcelWriter(full_path_export_output_data) as writer:
    df_output.to_excel(writer, sheet_name='flows_node_states')
    df_system_cost_output.to_excel(writer, sheet_name='system_costs')
    df_LCOE.to_excel(writer, sheet_name='LCOE')
    df_carbon_price.to_excel(writer, sheet_name='carbon_price')

In [235]:
full_path_export_output_data#df_LCOE

'C:\\Users\\luc.eco\\OneDrive - CBS - Copenhagen Business School\\Documents\\GitHub\\Nord_H2ub\\Spine_Projects\\03_output_data\\01_basic_energy_model_outputs\\Output_exported_TEST.xlsx'

### Finish and Exit the process:

- after executing the cell you can go back to SpineToolbox
- ensure that the data connection refers to the correct file

In [ ]:
avada_kedavra()